<a href="https://colab.research.google.com/github/jpedrozac/BASES-DE-DATOS-SEG-SEMESTRE/blob/main/2_lenguaje_humano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Primeros pasos con los idiomas**
La búsqueda de texto completo es una batalla entre la precisión (devolver la menor cantidad posible de documentos irrelevantes) y la memoria (devolver la mayor cantidad posible de documentos relevantes).

Se pueden implementar muchas tácticas para abordar la precisión y la memoria, como la modificación de palabras: p. busque "saltando", "salta" y "saltó" reduciendo las palabras a su raíz (raíz) - "saltar".

Sin embargo, el primer paso es identificar palabras usando un analizador.

* **Tokenize texto en palabras individuales:**
Los veloces zorros marrones → [Los, veloces, zorros, marrones]
* **Lowercase tokens:**
Los → los
* **Stem tokens a su forma raíz:**
zorros → zorro
* **Remover StopWords:**
[Los, veloces, zorros, marrones] → [veloces, zorros, marrones]



In [ ]:
#instalamos las librerias de ElasticSearch
!pip install elasticsearch
!pip install elasticsearch-dsl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 477.5/477.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 2.0 MB/s eta 0:00:00


In [ ]:
# Importar el cliente desde el módulo 'elasticsearch'
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q
from elasticsearch.exceptions import NotFoundError, RequestError
from pprint import pprint
import json

In [ ]:
# agregue su string de conexión
CLOUD_ID = "51ba68e5b9a4413e9a86103666932c48:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyRiYmEyYzlkZWJhYjc0YjJhOThjZjVlMTQ2N2E4YTY3MiQwMGVhZTM4MDU5NTQ0MTIwYjY1NjdjMTExYWJhMDI4Mg=="
es = Elasticsearch(
    cloud_id=CLOUD_ID,
    http_auth=("elastic", "iHa85Y0HRHo4eEIsBvsBB5z7")
)

<ipython-input-3-a5948f97f838>:3: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [ ]:
# La sisguiente instrucción muestra los datos de la instalación de elastic
# Obtener la información del servidor
info = es.info()

# Extraer y mostrar solo la versión
version = info.get('version', {}).get('number')
print("Elasticsearch version:", version)


Elasticsearch version: 8.13.4


In [ ]:
# Borrar el índice si existe
index_name = 'my_index'
try:
    response = es.indices.delete(index=index_name)
    print(f"Índice '{index_name}' borrado:", response)
except NotFoundError:
    print(f"Índice '{index_name}' no existe, por lo tanto no se borró.")
except RequestError as e:
    if e.error == 'index_not_found_exception':
        print(f"Índice '{index_name}' no encontrado, por lo tanto no se borró.")
    else:
        print(f"Error al intentar borrar el índice '{index_name}':", e.info)

Índice 'my_index' borrado: {'acknowledged': True}


# Creamos un Índice con Stemming:

Se define un analizador personalizado spanish_analyzer que incluye un filtro **spanish_stemmer** para reducir las palabras a sus raíces.
El campo title está configurado para usar este analizado

In [ ]:

index_template = {
    "settings": {
        "analysis": {
            "analyzer": {
                "spanish_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "spanish_stop",
                        "spanish_stemmer"
                    ]
                }
            },
            "filter": {
                "spanish_stop": {
                    "type": "stop",
                    "stopwords": "_spanish_"
                },
                "spanish_stemmer": {
                    "type": "stemmer",
                    "language": "light_spanish"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "title": {
                "type": "text",
                "analyzer": "spanish_analyzer"
            }
        }
    }
}

In [ ]:
# Se crea un indece segun la plantilla creada
es.indices.create(index='my_index', body=index_template)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

Vamos a insertar datos en nuestro indice

In [ ]:
documents = [
    { "title": "Estoy feliz por este zorro" },
    { "title": "No estoy contento con mi problema del zorro" },
    { "title": "Los zorros son astutos y rápidos" },  # Documento con "zorros"
    { "title": "Los perros no están contentos hoy" },
    { "title": "Los gatos son felices en el sol" }
]

# Indexar los documentos
for i, doc in enumerate(documents):
    es.index(index='my_index', body=doc, id=i+1)

##Consultas con elasticsearch-dsl


In [ ]:
# Realizar una búsqueda
s = Search(using=es, index='my_index')
q = Q('match', title='zorros')
s = s.query(q)
res = s.execute()

# Imprimir los resultados
for hit in res:
    print(hit.meta.id, hit.title, ' - Score:', hit.meta.score)

1 Estoy feliz por este zorro  - Score: 0.6103343
2 No estoy contento con mi problema del zorro  - Score: 0.52369374
3 Los zorros son astutos y rápidos  - Score: 0.52369374


Tenga en cuenta que ambos hits no contienen la palabra **zorros**, pero obtuvimos resultado con la palabra zorro.

Este enfoque asegura que las búsquedas sean más flexibles y encuentren documentos que contienen diferentes formas de una palabra basada en su raíz.

##  Normalizando Tokens
Dividir el texto en tokens es solo la mitad del trabajo. Para que esos tokens se puedan buscar más fácilmente, deben pasar por un proceso de normalización para eliminar las diferencias insignificantes entre palabras idénticas, como mayúsculas y minúsculas. Tal vez también necesitemos eliminar las diferencias significativas, para hacer que esta, ésta y está sean todas buscables como la misma palabra. ¿Buscarías un déjà vu, o simplemente un deja vu?

Este es el trabajo de los filtros de tokens, que reciben un flujo de tokens del tokenizador. Puede tener múltiples filtros de token, cada uno haciendo su trabajo particular. Cada uno recibe el nuevo flujo de token como resultado del filtro de token anterior.

* **Tipos de Tokenizadores:**
	- Standard
	- Simple
	- Whitespace
	- Stop
	- Keyword
	- Pattern
	- Fingerprint

In [ ]:
# Creamos un array con los diferentes analizadores que existen en Elastic.
analyzer = ['standard','simple','whitespace','stop','keyword','pattern','fingerprint','english']

#Imprimimos los diferentes analizadores con el mismo texto
for analize in analyzer:
    res = es.indices.analyze(body={
        "analyzer" : analize,
        "text" : ["HOLA MUNDO. ¡¡¡Hoy es el 2nd día de la semana!!!! es lunes. Por favor envíame un correo electrónico a friend@ucentral.edu.co"]

    })
    print("*****",analize,"*****")
    for i in res['tokens']:
        print(i['token'])
    print("\n")

***** standard *****
hola
mundo
hoy
es
el
2nd
día
de
la
semana
es
lunes
por
favor
envíame
un
correo
electrónico
a
friend
ucentral.edu.co


***** simple *****
hola
mundo
hoy
es
el
nd
día
de
la
semana
es
lunes
por
favor
envíame
un
correo
electrónico
a
friend
ucentral
edu
co


***** whitespace *****
HOLA
MUNDO.
¡¡¡Hoy
es
el
2nd
día
de
la
semana!!!!
es
lunes.
Por
favor
envíame
un
correo
electrónico
a
friend@ucentral.edu.co


***** stop *****
hola
mundo
hoy
es
el
nd
día
de
la
semana
es
lunes
por
favor
envíame
un
correo
electrónico
friend
ucentral
edu
co


***** keyword *****
HOLA MUNDO. ¡¡¡Hoy es el 2nd día de la semana!!!! es lunes. Por favor envíame un correo electrónico a friend@ucentral.edu.co


***** pattern *****
hola
mundo
hoy
es
el
2nd
d
a
de
la
semana
es
lunes
por
favor
env
ame
un
correo
electr
nico
a
friend
ucentral
edu
co


***** fingerprint *****
2nd a correo de dia el electronico enviame es favor friend hola hoy la lunes mundo por semana ucentral.edu.co un


***** english *****

##  Stop Words
Para usar stopwords personalizadas junto con el analizador estándar, todo lo que tenemos que hacer es crear una versión configurada del analizador y pasar la lista de stopwords que necesitamos:

In [ ]:
# Borrar el índice si existe
index_name = 'my_index'
try:
    response = es.indices.delete(index=index_name)
    print(f"Índice '{index_name}' borrado:", response)
except NotFoundError:
    print(f"Índice '{index_name}' no existe, por lo tanto no se borró.")
except RequestError as e:
    if e.error == 'index_not_found_exception':
        print(f"Índice '{index_name}' no encontrado, por lo tanto no se borró.")
    else:
        print(f"Error al intentar borrar el índice '{index_name}':", e.info)

Índice 'my_index' borrado: {'acknowledged': True}


Se define un analizador personalizado **spanish_analyzer** que incluye un filtro de stopwords personalizado custom_spanish_stop con las palabras "y" y "la".

In [ ]:
index_template = {
    "settings": {
        "analysis": {
            "analyzer": {
                "spanish_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "custom_spanish_stop",
                        "spanish_stemmer"
                    ]
                }
            },
            "filter": {
                "custom_spanish_stop": {
                    "type": "stop",
                    "stopwords": ["y", "los"]
                },
                "spanish_stemmer": {
                    "type": "stemmer",
                    "language": "light_spanish"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "title": {
                "type": "text",
                "analyzer": "spanish_analyzer"
            }
        }
    }
}

In [ ]:
# Se crea un indece segun la plantilla creada
es.indices.create(index='my_index', body=index_template)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

In [ ]:
# prueba con el analyzer standard
text = "Los zorros son astutos y rápidos"

analyzer = "standard"
response = es.indices.analyze(index='my_index', body={
    'analyzer': analyzer,
    'text': text
})

tokens = [token['token'] for token in response['tokens']]

# Imprimimos los tokens analizados
print(tokens)

['los', 'zorros', 'son', 'astutos', 'y', 'rápidos']


In [ ]:
# prueba con el analyzer personalizado
text = "Los zorros son astutos y rápidos"

analyzer = "spanish_analyzer"
response = es.indices.analyze(index='my_index', body={
    'analyzer': analyzer,
    'text': text
})

tokens = [token['token'] for token in response['tokens']]

# Imprimimos los tokens analizados
print(tokens)

['zorr', 'son', 'astut', 'rapid']


Actualizar stopwords es más fácil si las especifica en un archivo con el parámetro stopwords_path. Simplemente puede actualizar el archivo (en cada nodo del clúster) y luego obligar a que los analizadores se vuelvan a crear mediante cualquiera de estas acciones:

Por supuesto, actualizar la lista de stopwords no cambiará ningún documento que ya haya sido indexado. Se aplicará únicamente a las búsquedas a los documentos nuevos o actualizados. Para aplicar los cambios a los documentos existentes, deberá volver a indexar sus datos. [Consulte Reindexación de sus datos.](https://www.elastic.co/guide/en/elasticsearch/guide/master/reindex.html)

##  Correción Ortografia
La corrección ortográfica en Elasticsearch es un proceso que permite corregir automáticamente los errores de ortografía en los términos de búsqueda o en los términos indexados en el índice. Esto es especialmente útil cuando los usuarios ingresan consultas de búsqueda con errores de ortografía comunes.

In [ ]:
# Borrar el índice si existe
index_name = 'my_index'
try:
    response = es.indices.delete(index=index_name)
    print(f"Índice '{index_name}' borrado:", response)
except NotFoundError:
    print(f"Índice '{index_name}' no existe, por lo tanto no se borró.")
except RequestError as e:
    if e.error == 'index_not_found_exception':
        print(f"Índice '{index_name}' no encontrado, por lo tanto no se borró.")
    else:
        print(f"Error al intentar borrar el índice '{index_name}':", e.info)

Índice 'my_index' borrado: {'acknowledged': True}


Se define un analizador personalizado spanish_analyzer que incluye un filtro de corrección ortográfica **spanish_autocorrect**.

In [ ]:
index_template =   {
    "settings": {
        "analysis": {
            "analyzer": {
                "spanish_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "custom_spanish_stop",
                        "spanish_stemmer",
                        "spanish_autocorrect"
                    ]
                }
            },
            "filter": {
                "custom_spanish_stop": {
                    "type": "stop",
                    "stopwords": ["y", "la"]
                },
                "spanish_stemmer": {
                    "type": "stemmer",
                    "language": "light_spanish"
                },
                "spanish_autocorrect": {
                    "type": "hunspell",
                    "locale": "es_ES",
                    "dedup": True
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "title": {
                "type": "text",
                "analyzer": "spanish_analyzer"
            }
        }
    }
}

Para esta funcionalidad, necesita descargar e instalar el diccionario **Hunspell** para el Español. Esta funcionalidad la puede probar con la versión local de ElasticSearch